In [6]:
"""
生体医工学可視化（WordCloud + Termextract）
requirements:
 pip install pandas janome nlpplot matplotlib plotly
"""
import pandas as pd
import nlplot
import plotly
from plotly.subplots import make_subplots
from plotly.offline import iplot
import matplotlib.pyplot as plt
from pprint import pprint

from termextract import LRValue
from termextract.preprocessing import MeCabTokenizer

In [7]:
file_path = "datasets/生体医工学会論文5755件.csv"
df = pd.read_csv(file_path,encoding='utf-8')
df.head()

,No,Title,Author,Journal,Vol,No.1,Pages,Published_Year
0,0,リレー随筆他,NaN,生体医工学,43,4,800-802,2005
1,1,高齢者の使用を考慮した引戸の操作性の計測・評価のための実物大実験装置の開発と評価,"田中 眞二, 前川 陽, 奥野 竜平, 赤澤 堅造",生体医工学,43,4,795-799,2005
2,2,足指マウス開発にむけての基礎的検討―母指の運動特性―,"田中 則子, 上田 知生, 中尾 恵, 佐藤 哲大, 湊 小太郎, 吉田 正樹, 纐纈 和美",生体医工学,43,4,790-794,2005
3,3,血液検査用マイクロ流体デバイスの作製,"寺田 信幸, 山田 博之, 吉田 善一",生体医工学,43,4,785-789,2005
4,4,複数皮質活動の動的イメージングのためのfMRI-MEG統合解析法,"隠浪 康行, 小林 哲生, 鄭 址旭, 大橋 俊平, 濱田 昌司, 長峯 隆, 福山 秀直,...",生体医工学,43,4,777-784,2005


In [8]:
df["Published_Year"].unique()

array([2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023])

In [9]:
years= df["Published_Year"].unique().tolist()
n_years = len(years) # 19年間
print(n_years)

19


In [10]:
y1, y2 = 2005, 2007
df1 = df[(df['Published_Year']>=y1) & (df['Published_Year']<=y2)]
titles =  df1.loc[:,['Title']].values
pprint(titles[:10])

array([['リレー随筆他'],
       ['高齢者の使用を考慮した引戸の操作性の計測・評価のための実物大実験装置の開発と評価'],
       ['足指マウス開発にむけての基礎的検討―母指の運動特性―'],
       ['血液検査用マイクロ流体デバイスの作製'],
       ['複数皮質活動の動的イメージングのためのfMRI-MEG統合解析法'],
       ['低侵襲心臓外科手術用リアルタイム三次元超音波Integral Videography'],
       ['業務スケジューリングからみた看護業務属性の検討'],
       ['指タップ加速度計側システムの開発とパーキンソン病診断支援への応用'],
       ['パラレルワイヤ駆動システムと繰返し学習制御を用いた生体部位の他動的軌道追従の実験的検証'],
       ['LAMP法と導電性中空糸膜を利用したウイルス検査法におけるウシ血清個体差の影響']], dtype=object)


In [12]:
docs = {}
for y in years:
    docs[y] = df[(df['Published_Year']==y)]['Title'].values.tolist()

In [13]:
tok = MeCabTokenizer() 
ignores = ['研究','開発','検討','試作','提案','紹介','展望','事例','課題',"および", "評価"]
ignores1 = ['型']
ignores2 = ['中','的','時','システム','用']

compound_nouns = []
for y in years:
    for sentence in docs[y]:
        if sentence.isascii():
            continue 
    
        tokens = tok.iter_token(sentence)
        nouns = tok.extract_nouns(tokens)
        # print(nouns)
        nouns = tok.filter_nouns(nouns, n=1, stopwords=ignores, stopwords_begin=ignores1, stopwords_end=ignores2)
        # print(nouns)
        if len(nouns) > 0 :
            compound_nouns.extend(nouns)

lr = LRValue()
# lr_values = lr.fit_transform(compound_nouns)
lr_values = lr.fit(compound_nouns).transform(compound_nouns, mode=3)
lr_values = [(k,v) for k,v in lr_values.items() if v>0]
data_collection = sorted(lr_values, key=lambda x:x[1], reverse=True)
list(data_collection)[:50]

/tmp/working/notebooks/theme_river/termextract/termextract.py:53: RuntimeWarning:

invalid value encountered in power



[('性', 104.69479452198185),
 ('計測', 79.87490219086344),
 ('画像 計測', 71.09330215402213),
 ('機能 計測', 69.24706411841348),
 ('細胞', 67.6904720030818),
 ('細胞 機能', 63.74703163941054),
 ('画像', 63.2771680782255),
 ('細胞 運動', 63.15513972885417),
 ('計測 解析', 61.48471641170195),
 ('細胞 モデル', 61.25183078345497),
 ('計測 光', 60.61555494078824),
 ('光 計測', 60.61555494078824),
 ('機能', 60.03332407921454),
 ('脳 機能 計測', 59.93529742122243),
 ('生体 画像 計測', 59.754471259142555),
 ('運動 機能', 59.475913985403544),
 ('超音波 画像 計測', 59.44879840197806),
 ('運動', 58.92367945062494),
 ('生体 機能 計測', 58.715422927581805),
 ('生体 計測', 58.06736174869173),
 ('画像 刺激', 57.19700083997363),
 ('刺激 画像', 57.19700083997363),
 ('細胞 組織', 56.689450737053555),
 ('細胞 解析', 56.60121786151975),
 ('光 脳 機能 計測', 56.098526812357065),
 ('モデル', 55.42562584220407),
 ('血管 細胞', 53.99452807653034),
 ('呼吸 機能 計測', 53.76080874400938),
 ('自動 計測', 53.56148350403636),
 ('細胞 センサ', 52.00223301501167),
 ('脳 機能', 51.918104413807754),
 ('刺激', 51.70106381884226),
 ('生体 画像'

In [44]:
import random
import re
from collections import defaultdict

topics = [
# common
    '骨','筋', '脳', '細胞','生体','血管','呼吸','計測','画像','超音波','人工'  # uni-gram
    # '脳 機能', '生体 機能', '画像 計測', '光 刺激', '超音波 画像' # bi-gram
    # '診断 用 超音波', '脳 機能 解析', '細胞 動態 計測' #tri-gram 
]

base = topics

random.shuffle(base)

d = 3
x = years[d:len(years)]
Y = []
for i in range(len(years)-d):
    y1, y2 = years[i], years[i+2] 
    print(f"{y1}-{y2}")
    compound_nouns = []
    for y in range(y1, y2+1):
        for sentence in docs[y]:
            if sentence.isascii():
                continue 
        
            tokens = tok.iter_token(sentence)
            nouns = tok.extract_nouns(tokens)
            # print(nouns)
            nouns = tok.filter_nouns(nouns, n=1, stopwords=ignores, stopwords_begin=ignores1, stopwords_end=ignores2)
            # print(nouns)
            if len(nouns) > 0 :
                compound_nouns.extend(nouns)

    lr = LRValue()
    lr_values = lr.fit(compound_nouns).transform(compound_nouns, mode=3)
    lr_values = [(k,v) for k,v in lr_values.items() if v>0]
    data = sorted(lr_values, key=lambda x:x[1], reverse=True)

    freq = lr.compound_noun
    target_freq = defaultdict(int)
    target_item = defaultdict(list)
    target_freq = {key:0 for key in topics}
    for w, c in freq.items():
        for k in topics:
            if re.match(r'.*'+str(k), w):
                target_freq[k] += 1
                target_item[k] += [w]

    # pprint(data[:5])
    pprint(list(target_freq.items())[:5])
    # pprint(list(target_item.items())[:5])
    Y += [target_freq.values()]
print(x)
print(Y[0:2]) 

2005-2007
[('脳', 15), ('計測', 19), ('筋', 11), ('骨', 15), ('細胞', 6)]
2006-2008
[('脳', 13), ('計測', 15), ('筋', 9), ('骨', 22), ('細胞', 19)]
2007-2009
[('脳', 21), ('計測', 15), ('筋', 14), ('骨', 19), ('細胞', 22)]
2008-2010
[('脳', 19), ('計測', 15), ('筋', 18), ('骨', 18), ('細胞', 22)]
2009-2011
[('脳', 18), ('計測', 17), ('筋', 21), ('骨', 10), ('細胞', 10)]
2010-2012
[('脳', 12), ('計測', 20), ('筋', 19), ('骨', 3), ('細胞', 6)]
2011-2013
[('脳', 16), ('計測', 24), ('筋', 17), ('骨', 7), ('細胞', 14)]
2012-2014
[('脳', 24), ('計測', 33), ('筋', 19), ('骨', 9), ('細胞', 25)]
2013-2015
[('脳', 53), ('計測', 63), ('筋', 45), ('骨', 17), ('細胞', 68)]
2014-2016
[('脳', 64), ('計測', 90), ('筋', 64), ('骨', 25), ('細胞', 99)]
2015-2017
[('脳', 73), ('計測', 105), ('筋', 73), ('骨', 35), ('細胞', 111)]
2016-2018
[('脳', 58), ('計測', 93), ('筋', 67), ('骨', 42), ('細胞', 93)]
2017-2019
[('脳', 77), ('計測', 102), ('筋', 67), ('骨', 38), ('細胞', 89)]
2018-2020
[('脳', 80), ('計測', 88), ('筋', 66), ('骨', 37), ('細胞', 86)]
2019-2021
[('脳', 91), ('計測', 96), ('筋', 59), ('骨', 

In [45]:
from  matplotlib import rcParams
rcParams['font.family'] = 'Noto Sans CJK JP'

In [46]:
fig = plt.figure(figsize=[15,6], facecolor='lightblue')
y_axis = [ [list(t)[i] for t in Y ] for i,_ in enumerate(base)]

ax = []
n_fig = 1

for i in range(n_fig):
    ax.append(fig.add_subplot(n_fig,1,i+1) )

d = 9

# Color patterns for keywords in stackplot
# cf. https://saruwakakun.com/design/gallery/palette

# pattern 1: system default
colors1 = None  # default colors

# pattern 2: dark
colors2=['#2E2E2E', '#626262', '#FF5722', '#FF9800', '#FFC107', 
        '#204495', '#6495ED', '#1C75BC', '#5CC0EF']

# pattern 3 : light
colors3=['#F6E5D7', '#73D0C2', '#ED8D49', '#FFC530', '#978D86',
        '#E85A70', '#4EA1D5', '#AAA5D1', '#454552']

for i in range(n_fig):
    labels = list(base)[i*d:(i+1)*d]
    ax[i].stackplot(x, *(y_axis[i*d:(i+1)*d]), labels=labels, colors=colors2, baseline='wiggle')
    # ax[i].legend(loc='best')
    ax[i].legend(bbox_to_anchor=(1.03, 1.0), loc='upper left')
    # ax[i].set_xlabel('年度')
    # ax[i].set_ylabel('使用頻度')

plt.savefig('uni-gram.png')
plt.show()